# Special Flood Inundation Mapping

This notebook demonstrates how to create special maps with a FLDPLN library. Those maps include:
* Flood depth map with a constant stage at all the flood source pixels (FSPs, i.e., stream pixels)
* 'MinDtf' map, the minimum stage (or depth to flood (DTF)) needed to inundate a floodplain pixel (FPP)
* 'NumOfFsps' map, the number of FSPs that may inundate a FPP
* 'Depression' map, the filled depression depth 

## Import Modules and Packages

In [8]:
import time

# Importing DASK libraries for using DASK for parallel mapping
from dask.distributed import Client, LocalCluster
from dask import visualize

# import the mapping module
from fldpln.mapping import *

## Setup Tiled Library and Output Map Folders

Here we setup the folder under which tiled libraries (organized as sub-folders) are located. We also setup the output folder under which a map sub-folder and a 'scratch' sub-folder will be created. The map folder, which is specified later, contains all inundation depth maps for the libraries. The scratch folder stores temporary files during the mapping. We can create the special maps for a set of tiled libraries and we assume that those libraries are located under the same folder. It is such designed as **libraries may overlap in space**, and tiles can be uniquely identified by the combination of library name and tile ID.

In [9]:
# tiled library folder that may have more than one tiled libraries
libFolder =  'E:/fldpln/sites/wildcat_10m_3dep' # wildcat

# Set output folder
outputFolder = 'E:/fldpln/sites/wildcat_10m_3dep/maps'

## Setup Mapping Parameters

There are several parameters need to be set before the maps are generated. Those include:
* The tiled libraries to be mapped (see variable libs2Map)
* What special map to be created (i.e., specialDof)
* The name of the special map folder (i.e., outMapFolderName) which is under the output folder and stores all inundation depth maps as COG GeoTIFFs
* Whether to mosaic the tiles as single COG GeoTIFF file and whether to use a Dask local cluster for parallel mapping

In [11]:
# select the libraries under tiled library folder to generate maps
libs2Map = ['tiledlib']

# set the special map wanted
# 'MinDtf','NumOfFsps','Depression', or a real number (for example 20.5, in foot for KS libraries) representing constant DOF at all the FSPs
specialDof = 'MinDtf' # Wildcat 3DEP DEM has a vertical unit of meters

# set up output map-folder under the outputFolder
outMapFolderName = 'mindtf'

# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)

# whether mosaic tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False
# numOfWorkers = round(0.8*os.cpu_count())

## Generate Special Map

In [ ]:

# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# libs to map
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        MosaicGtifs(outMapFolder,tileTifs,mosaicTifName,keepTifs=False)
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False

## Visualize and Examine the Special Maps

Here we can visualize the special maps generated above using GIS software such as ArcGIS Pro to better understand them.